<a href="https://colab.research.google.com/github/cheburyshkova/declarationanalytics/blob/main/declarationanalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Обработка данных

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install pymorphy2

In [161]:
import pandas as pd
from typing import List
import re
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords

In [138]:
data = pd.read_csv("product_data.csv")

#data.head()

In [162]:

def tokenize_row(row: pd.Series) -> pd.Series:

    tokens_manufacturer_address = word_tokenize(row['Manufacturer Address'])

    tokens_generic_product_name = word_tokenize(row['Generic Product Name']) if 'Generic Product Name' in row else []

    return pd.Series([tokens_generic_product_name, tokens_manufacturer_address])

#data[['Tokens_Generic_Product_Name', 'Tokens_Manufacturer_Address']] = data.apply(tokenize_row, axis=1)
#data[['Generic Product Name', 'Tokens_Generic_Product_Name', 'Manufacturer Address', 'Tokens_Manufacturer_Address']].head()

In [140]:
countries_list = [
    'япония', 'китай', 'беларусь', 'соединенные штаты', 'иран', 'турция',
    'россия', 'германия', 'франция', 'италия', 'испания',
    'соединенное королевство великобритании', 'греция', 'бельгия',
    'вьетнам', 'бразилия', 'гонконг', 'доминиканская республика',
    'индонезия', 'ирландия', 'финляндия', 'корея', 'узбекистан',
    'тайвань', 'пакистан', 'польша', 'соединенное королевство', 'австрия', 'чили', 'швейцария,', 'швеция', 'португалия', 'нидерланды',
    'мексика', 'китайская народная республика'
]

In [ ]:
def extract_country_from_address(address: str, countries_list: List[str]) -> str:

    for country in countries_list:
        if country.lower() in address.lower():
            return country
    return "неизвестная страна"

data['Country'] = data['Manufacturer Address'].apply(lambda x: extract_country_from_address(x, countries_list))

data[['Manufacturer Address', 'Country']].head()

In [142]:
def clean_text(text: str) -> str:

    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[a-zA-Z0-9]', '', text)
    return text

data['Cleaned_Product_Name'] = data['Generic Product Name'].apply(clean_text)

#data[['Generic Product Name', 'Cleaned_Product_Name']].head()

In [143]:
morph = MorphAnalyzer()

def lemmatize_text(text: str, morph_analyzer: MorphAnalyzer) -> str:

    try:
        tokens = word_tokenize(text)
        lemmatized_tokens = [morph_analyzer.parse(word)[0].normal_form for word in tokens]
    except Exception as e:
        print(f"Ошибка при лемматизации: {e}")
        return text
    return ' '.join(lemmatized_tokens)

data['Lemmatized'] = data['Cleaned_Product_Name'].apply(lambda x: lemmatize_text(x, morph))

#data[['Lemmatized']].head()

In [ ]:
def calculate_total_word_frequency(data: pd.DataFrame, column_name: str, top_n: int) -> pd.DataFrame:

    try:
        big_text = ' '.join(data[column_name].astype(str))
        tokens = word_tokenize(big_text)
        word_freq = Counter(tokens)
        freq_df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency']).sort_values(by='Frequency', ascending=False)
    except Exception as e:
        print(f"Ошибка при вычислении частоты слов: {e}")
        return pd.DataFrame(columns=['Word', 'Frequency'])
    return freq_df.head(top_n)

total_word_frequency = calculate_total_word_frequency(data, 'Lemmatized', 10)

top_words = total_word_frequency['Word'].tolist()

print(top_words)

In [ ]:
def extract_adjectives(words: List[str]) -> List[str]:

    morph = pymorphy2.MorphAnalyzer()
    adjectives = [word for word in words if 'ADJF' in morph.parse(word)[0].tag or 'ADJS' in morph.parse(word)[0].tag]
    return adjectives

adjectives_list = extract_adjectives(top_words)

print(adjectives_list)

In [ ]:
def extract_prepositions_and_conjunctions(words: List[str]) -> List[str]:

    morph = pymorphy2.MorphAnalyzer()
    prepositions_and_conjunctions = [word for word in words if 'PREP' in morph.parse(word)[0].tag or 'CONJ' in morph.parse(word)[0].tag]
    return prepositions_and_conjunctions

prepositions_and_conjunctions_list = extract_prepositions_and_conjunctions(top_words)

print(prepositions_and_conjunctions_list)

In [ ]:
def extract_verbs(words: List[str]) -> List[str]:

    morph = pymorphy2.MorphAnalyzer()
    verbs = [word for word in words if 'VERB' in morph.parse(word)[0].tag or 'INFN' in morph.parse(word)[0].tag]
    return verbs

verbs_list = extract_verbs(top_words)

print(verbs_list)

In [148]:
def remove_stop_words(text: str, stop_words: list) -> str:

    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

stop_words_list = ['в', 'из', 'с', 'для','и', 'до', 'на', 'или', 'за', 'от', 'тот', 'упаковка', 'марка','артикул', 'модель',  'общий', 'свежий', 'искусственный', 'предмет', 'си', 'набор', 'год', 'номинальный', 'г', 'они', 'назначение', 'кожа', 'старый', 'материал', 'элемент', 'го', 'он', 'к', 'число', 'в_', 'отдельный', 'успокаивать', 'верхний', 'мужской', 'летний', 'то', 'под', 'работать', 'сочетание', 'конкест', 'гдм', 'пикет', 'ла', 'корк', 'номер', 'месяц', 'индекс', 'лежание', 'товарный', 'знак', 'транспортный', 'промышленность', 'технологический', 'промышленный', 'верхом', 'ирландский', 'вест', 'каск', 'сочетание', 'мпа', 'марк', 'расчётный', 'тип', 'не','второй', 'происхождение', 'слой', 'готовый', 'пищевой', 'происхождение', 'ребёнок', 'женщина', 'сойка', 'поставлять', 'отдельный','белый', 'световой', 'серия', 'тёплый', 'зелёный', 'средство', 'уход',
'экстракт', 'эссенция', 'концентрат', 'стать', 'отдельный', 'оборудование','синтетический', 'маркировка', 'покрытие', 'скорость', 'гвладивосток', 'ултигровый', 'край', 'цель', 'натуральный', 'собственный', 'изготовление', 'февраль', 'выпуск', 'обрабатывать', 'центр', 'компонент', 'мужчина', 'послепродажный', 'изделие', 'отдельный','торговый', 'товарный', 'знак', 'часть', 'качество', 'взрослый', 'дорожный', 'ип', 'л', 'смешанный']

data['Product_Name'] = data['Lemmatized'].apply(lambda x: remove_stop_words(x, stop_words_list))

#data[['Product_Name']].head()

In [149]:
def keep_first_five_words(text):

    words = text.split()
    return ' '.join(words[:5])

data['Products'] = data['Product_Name'].apply(keep_first_five_words)

#data[['Products']].head()

In [154]:
dataset = data[['Products', 'Country']]
dataset.to_csv('dataset.csv', index=False)

In [155]:
df = pd.read_csv('dataset.csv', encoding='utf-8')

In [156]:
df.dropna(subset=['Products'], inplace=True)

In [157]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Products'])

In [ ]:
num_clusters = 7
kmeans = KMeans(n_clusters=num_clusters, random_state=143)
df['Cluster'] = kmeans.fit_predict(X)

In [ ]:
terms = vectorizer.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

top_words = []
for i in range(num_clusters):
    cluster_words = [terms[ind] for ind in order_centroids[i, :5]]
    top_words.append(cluster_words)


for i, words in enumerate(top_words):
    print(f"Cluster {i+1}: {', '.join(words)}")